# Hydrofabric Geopackage Viewer

Read geopackage with geopandas.  Get list of layers, loop and read all layers and store in a dictionary.


Requires:
- To be run on machine with access to https://hydroapi.oe.nextgenwaterprediction.com/
- `.env` with AWS credentials

In [ ]:
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
from dotenv import load_dotenv
from ipywidgets import interact

from icefabric_tools.geo.geo_utils import create_time_series_widget, get_geopackage_uri, get_observational_uri

In [2]:
# before running this cell, make sure you created a .env file in this directory with your AWS credentials in it
load_dotenv()

True

## Getting streamflow observations for different gages


Step 1) getting the data

In [3]:
streamflow_obs_uri = get_observational_uri(gage_id="01013500")
streamflow_obs_df = pd.read_csv(streamflow_obs_uri)
streamflow_obs_df

,dateTime,q_cms
0,1990-10-01 05:00:00,19.623575
1,1990-10-01 06:00:00,20.416447
2,1990-10-01 07:00:00,21.237635
3,1990-10-01 08:00:00,21.860605
4,1990-10-01 09:00:00,22.285358
...,...,...
297303,2024-08-30 20:00:00,13.280601
297304,2024-08-30 21:00:00,13.280601
297305,2024-08-30 22:00:00,13.280601
297306,2024-08-30 23:00:00,13.280601


Step 2) streamflow visualization

In [5]:
streamflow_obs_df["dateTime"] = pd.to_datetime(streamflow_obs_df["dateTime"])

# Create sliders
start_slider = widgets.SelectionSlider(
    options=streamflow_obs_df["dateTime"].dt.strftime("%Y-%m-%d %H:%M:%S").tolist(),
    description="Start:",
    layout=widgets.Layout(width="95%"),
)

end_slider = widgets.SelectionSlider(
    options=streamflow_obs_df["dateTime"].dt.strftime("%Y-%m-%d %H:%M:%S").tolist(),
    description="End:",
    layout=widgets.Layout(width="95%"),
)

# Scatter Plot of observations
create_time_series_widget(streamflow_obs_df, start_slider, end_slider, point_size=5)

interactive(children=(SelectionSlider(description='Start:', layout=Layout(width='95%'), options=('1990-10-01 0…

## Geopackage Summary

Print list of layers, number of catchments, and list of hydrolocations.

In [10]:
geopackage_uri

's3://ngwpc-hydrofabric-oe/2.1/CONUS/06710385/GEOPACKAGE/USGS/2024_Dec_16_20_29_12/gauge_06710385.gpkg'

In [15]:
geopackage_uri = get_geopackage_uri(gage_id="06710385", version=2.2)

layers = gpd.list_layers(geopackage_uri).name.tolist()

layer_df = {}

for layer in layers:
    df = gpd.read_file(geopackage_uri, layer=layer)
    layer_df[layer] = df

layers_str = ", ".join(layers)
print("Layers:")
print(layers_str)
row, col = layer_df["divides"].shape
print("Number of catchments:")
print(row)
print("Hydrolocations:")
hl = layer_df["hydrolocations"].hl_uri.tolist()
hl_str = ", ".join(hl)
print(hl_str)

Layers:
flowpaths, divides, nexus, lakes, pois, hydrolocations, flowpath-attributes, flowpath-attributes-ml, network, divide-attributes
Number of catchments:
26
Hydrolocations:
huc12-101900020801, huc12-101900020802, gages-06710385


## Map of divides, nexuses, and flowpaths

Plot divides, nexuses, and flowpaths on a map.

In [13]:
divides = layer_df["divides"].to_crs(epsg=3857)

flowpaths = layer_df["flowpaths"].to_crs(epsg=3857)
nexus = layer_df["nexus"].to_crs(epsg=3857)

div_ex = divides.explore()
fl_ex = flowpaths.explore(m=div_ex, color="yellow")
nexus.explore(m=fl_ex, color="red")

## View Layers

Select layer and print table

In [14]:
def show_layer(layer_name):
    display(layer_df[layer_name])


interact(show_layer, layer_name=layers)

interactive(children=(Dropdown(description='layer_name', options=('flowpaths', 'divides', 'nexus', 'lakes', 'p…

<function __main__.show_layer(layer_name)>